### Luigi Module

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from jupyter_notebook import load_parameters 

pars = load_parameters()

input_file = pars.get('input')
output_file = pars.get('output')

# Dealing with multiple outputs

dataset = pars.get('dataset')

KeyError: 'PARS'

In [4]:
import pandas as pd

In [9]:
# input_file = '../../data/stack-overflow/pandas-solutioncode-h1h2'
# output_file = '../../data/stack-overflow/result1.txt'
# dataset = '../../data/stack-overflow/Dataset - Pandas.csv'
dataset = pd.read_csv(dataset, encoding='ISO-8859-1', error_bad_lines=False)

result_df = pd.read_pickle(input_file)


In [7]:
def buildTaggedDatasetDSForEvaluation(tagged_dataset_df):
    taggedDSEvalDict = dict()
    try:
        for index, row in tagged_dataset_df.iterrows():
            Id = row['AnswerId']
            if Id != 0:
                
                if Id in taggedDSEvalDict:
                    solutionList = taggedDSEvalDict[Id]
                else:
                    solutionList = list()                    
                lineList = list()
                lines = row['Solution'].split('\n')
                for line in lines:
                    if line.strip() != '':
                        pair = [line, False]
                        lineList.append(pair)
                solutionList.append(lineList)
                taggedDSEvalDict[Id]= solutionList
        
    except Exception as e:
        print(e)
    return taggedDSEvalDict


def buildResultSetForEvaluation(result_df):
    resultSetDict = dict()
    try:
        for index, row in result_df.iterrows():
            ansId = row['Id']
            solutionLinesTuple = list()                    
            predictedLines = row['Solution']
            for line in predictedLines:
                if line.strip() != '':
                    pair = [line, False]
                    solutionLinesTuple.append(pair)
            resultSetDict[ansId]= solutionLinesTuple
        
    except Exception as e:
        print(ansId, e)
    return resultSetDict

def markTheLines(taggedDSEvalDict, resultSetDict):
    for Id, pairs in resultSetDict.items():
        try:
            solutionList = taggedDSEvalDict[Id] # Error
            for respair in pairs:
                for solution in solutionList:
                    for tagpair in solution:
                        if tagpair[0].strip() == respair[0].strip():
                            tagpair[1] = True
                            respair[1] = True
        except KeyError:
            pass # Ignore it for now
    return taggedDSEvalDict, resultSetDict

In [13]:
def evaluate(result_df):
    TP = 0
    FP = 0
    taggedDSEvalDict =  buildTaggedDatasetDSForEvaluation(dataset)
    resultSetDict = buildResultSetForEvaluation(result_df)
    taggedDSEvalDict, resultSetDict = markTheLines(taggedDSEvalDict, resultSetDict)
    for Id, pairs in resultSetDict.items():
        try:
            MA = True
            for pair in pairs:
                MA = MA and pair[1]
            if MA:
                MP = False

                solutions = taggedDSEvalDict[Id]
                for solution in solutions:
                    MPS = True
                    for tagpair in solution:
                        MPS = MPS and tagpair[1]
                    MP = MP or MPS
                if MP:
                    TP = TP + 1
                else:
                    FP = FP + 1

            else:
                FP = FP + 1
        except KeyError:
            pass # Ignore it for now
            
    return float(TP)/(TP+FP)

precision = evaluate(result_df)
# precision = precision * 100
print(precision)

'float' object has no attribute 'split'
{7776679: [['N', False], ['A', False]], 7837722: [['N', False], ['A', False]], 8916302: [['N', False], ['A', False]], 8991709: [['N', False], ['A', False]], 9555635: [['N', False], ['A', False]], 9588331: [['N', False], ['A', False]], 9652832: [['N', False], ['A', False]], 9758450: [['N', False], ['A', False]], 9762935: [['N', False], ['A', False]], 9787853: [['N', False], ['A', False]], 9850954: [['N', False], ['A', False]], 10020591: [['N', False], ['A', False]], 10065051: [['N', False], ['A', False]], 10114399: [['N', False], ['A', False]], 10175068: [['N', False], ['A', False]], 10202570: [['N', False], ['A', False]], 10373660: [['N', False], ['A', False]], 10376647: [['N', False], ['A', False]], 10457584: [['N', False], ['A', False]], 10464738: [['N', False], ['A', False]], 10511024: [['N', False], ['A', False]], 10565282: [['N', False], ['A', False]], 10591000: [['N', False], ['A', False]], 10636024: [['N', False], ['A', False]], 10665889: 

In [6]:

with open(output_file + '.txt', 'w') as f:
    f.write('%.4f' % precision)
